**02_preprocess_for_ML**

- Filtered and prepped data for Machine learning
- main goal was feature reduction

used this container: 
ml:1.0

In [1]:
import logging
import pathlib
import os
import requests
import tempfile

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import scipy.stats as ss
from itertools import compress

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import VarianceThreshold, mutual_info_classif, SelectPercentile, SelectFromModel
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
if __name__ == "__main__":
    
    #this is for the outlier trimming
    
    class OutlierRemover(BaseEstimator,TransformerMixin):
        def __init__(self,factor=1.5):
            self.factor = factor

        def outlier_detector(self,X,y=None):
            X = pd.Series(X).copy()
            q1 = X.quantile(0.1)
            q3 = X.quantile(0.9)
            iqr = q3 - q1
            self.lower_bound.append(q1 - (self.factor * iqr))
            self.upper_bound.append(q3 + (self.factor * iqr))

        def fit(self,X,y=None):
            self.lower_bound = []
            self.upper_bound = []
            X.apply(self.outlier_detector)
            return self

        def transform(self,X,y=None):
            X = pd.DataFrame(X).copy()
            for i in range(X.shape[1]):
                x = X.iloc[:, i].copy()
                x.loc[x < self.lower_bound[i]] = self.lower_bound[i]
                x.loc[x > self.upper_bound[i]] = self.upper_bound[i]
                X.iloc[:, i] = x
            return X

    outlier_remover = OutlierRemover()


    
    # define functions for correlation categorical - this only selects the top 15 independent features  
    def correlation_cat(dataset, cram):
        results_df = pd.DataFrame()  # Set of all the names of correlated columns
        corr_matrix = cram
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                #if abs(corr_matrix.iloc[i, j]) < threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i] # getting the name of column
                indexname = corr_matrix.index[j] # getting the name of the index
                if (dataset[colname] == 'Not Reported').sum() <= (dataset[indexname] == 'Not Reported').sum():
                    results_df = results_df.append(pd.DataFrame([[colname, abs(corr_matrix.iloc[i, j])]], 
                                    columns=['colname', 'coeff']))
                else:
                    results_df = results_df.append(pd.DataFrame([[indexname, abs(corr_matrix.iloc[i, j])]], 
                                        columns=['colname', 'coeff']))
        # get the top 5 uncorrelated values
        results_df = results_df.sort_values(by='coeff', ascending=True).drop_duplicates(subset=['colname'], keep='last')['colname'][:30].tolist()
        # create list of columns to be excluded
        exclude_list = list(set(dataset.columns.tolist()) - set(results_df))
        return exclude_list
    
    # correlation numeric
    def correlation(dataset, threshold):
        col_corr = set()  # Set of all the names of correlated columns
        corr_matrix = dataset.corr()
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                    colname = corr_matrix.columns[i] # getting the name of column
                    indexname = corr_matrix.index[j] # getting the name of the index
                    if dataset[colname].isna().sum() <= dataset[indexname].isna().sum():
                        col_corr.add(indexname)
                    else:
                        col_corr.add(colname)                
        return col_corr

    def cramers_v(x, y):
        """ calculate Cramers V statistic for categorial-categorial association.
            uses correction from Bergsma and Wicher,
            Journal of the Korean Statistical Society 42 (2013): 323-328
        """
        confusion_matrix = pd.crosstab(x,y)
        chi2 = ss.chi2_contingency(confusion_matrix)[0]
        n = confusion_matrix.sum().sum()
        phi2 = chi2/n
        r,k = confusion_matrix.shape
        phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
        rcorr = r-((r-1)**2)/(n-1)
        kcorr = k-((k-1)**2)/(n-1)
        return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))
    
def preprocess_dataset(base_dir, executionid, suffix):
    df_model = pd.read_csv(f'{base_dir}raw_data/reprocessed_model_log.csv')
    file_name_base = str(df_model[df_model['ExecutionID'] == executionid][['aggregation_time','gap']].values[0][0]) + '_years_' + str(df_model[df_model['ExecutionID'] == executionid][['aggregation_time','gap']].values[0][1])
    
    # read in parquet
    df = pd.read_parquet(f'{base_dir}raw_data/preprocessed_data/ef_agg_before_biopsy_{file_name_base}_gap.parquet', engine='pyarrow')

    # drop NaN columns from parquet
    df = df.dropna(subset=['gfb_subject_id'])
    # convert datatype of patient_id
    df['gfb_subject_id'] = df['gfb_subject_id'].astype(str)

    # Since we get a headerless CSV file we specify the column names here.
    feature_columns_names = {'gfb_subject_id':str,
                             'place_of_origin':str,
                             'disease_simple':int,
                             'TBM_Thickening':str,
                             'GBM_Thickening':str,
                             'Inflammatory_Cell_Infiltrate':str,
                             'Nodular_Sclerosis':str,
                             'Mesangial_Hypercellularity':str,
                             'Arteriolar_Hyalinosis':str,
                             'source_subject_id':str,
                             'age_biopsy': float,
                             'diabetes_age': float,
                             'duration_dm':str,
                             'Diabetic_Retinopathy':str,
                             'is_DN':str,
                             'k_means': int,
                             'k_means_binary':str}


    feature_columns_names = ['gfb_subject_id',
                             'place_of_origin',
                             'disease_simple',
                             'TBM_Thickening',
                             'GBM_Thickening',
                             'Inflammatory_Cell_Infiltrate',
                             'Nodular_Sclerosis',
                             'Mesangial_Hypercellularity',
                             'Arteriolar_Hyalinosis',
                             'source_subject_id',
                             'age_biopsy',
                             'diabetes_age',
                             'duration_dm',
                             'Diabetic_Retinopathy',
                             'is_DN',
                             'k_means',
                             'k_means_binary']


    # read file + merge with other df to filter to 523 patients + drop patient ID
    df_target = pd.read_csv(f'{base_dir}{suffix}/ML_target_variables_trainval.csv')

    df_target_temp = df_target[['gfb_subject_id', 'age_biopsy', 'diabetes_age', 'duration_dm', 'Diabetic_Retinopathy']]

    df = df.merge(df_target_temp, on='gfb_subject_id', how='right')    
    df = df.drop(columns=['gfb_subject_id', 'subject_id', 'biopsy_date', 'dob', 'freq_of_visits', 'state', 'biopsy_indication'])

    #need tocoerce some columns into approriate datatype
    df['age_biopsy'] = df['age_biopsy'].astype(float)
    df['diabetes_age'] = df['diabetes_age'].astype(float)
    df['ruca_code'] = df['ruca_code'].astype(str)
    df['duration_dm'] = df['duration_dm'].fillna(value='Not Reported') # not sure why this came up

    # replace any infitiny values with nans
    df= df.replace([np.inf, -np.inf], np.nan)

    print('original', df.shape)

    #drop columns that have more than 75% NaNs
    df = df.dropna(axis='columns', thresh= round(df.shape[0]*0.25))

    print('after dropna', df.shape)

    # transform datatypes into either float or objects/str
    df = pd.concat([df.select_dtypes(include='float'), 
                    df.select_dtypes(include='int64').astype(float),
                    df.select_dtypes(include='object'), 
                    df.select_dtypes("bool").astype(str)], axis=1, ignore_index=False)

    #this is a dumb strategy to fill NAs in the categoriacal variables otherwise the pipeline below fails (only in Sagemaker, in the preprocessing NB it is fine), also need to force everything to str
    mask = df.applymap(type) == float
    d = {True: 'TRUE'}
    df = df.where(mask, df.fillna('Not Reported'))
    df = df.where(mask, df.replace(d))
    df = df.where(mask, df.astype(str))

    df_num = df.select_dtypes(include='float')
    df_cat = df.select_dtypes(include= 'object')

    print('number of numeric features', df_num.shape)
    print('number of categorical features', df_cat.shape)

    # drop features that are not that variable
    loc_filter = VarianceThreshold(threshold=(.8 * (1 - .8)))

    #correlation threshold 0.75
    corr_threshold = 0.75

    # eliminate correlated features by NaNs
    corr_features_num = list(correlation(df_num, corr_threshold))
    df_num = df_num.drop(corr_features_num ,axis=1)

    print('number of numeric features after correlation drop', df_num.shape)

    # this is to make the Cramer's V correlation matrix for the categorical features - seems overly complicated
    factors_paired = [(i,j) for i in df_cat.columns.values for j in df_cat.columns.values] 
    cram = []
    size_df = len(df_cat.columns.tolist())
    for f in factors_paired:
        if f[0] != f[1]:
            chitest = cramers_v(df_cat[f[0]], df_cat[f[1]])   
            cram.append(chitest)
        else:      # for same factor pair
            cram.append(0)
    cram = np.array(cram).reshape((size_df,size_df)) # shape it as a matrix
    cram = pd.DataFrame(cram, index=df_cat.columns.values, columns=df_cat.columns.values) # then a df for convenience

    corr_features_cat = correlation_cat(df_cat, cram)
    df_cat = df_cat.drop(corr_features_cat ,axis=1)

    print('number of categorical features after correlation drop', df_cat.shape)

    corr_features_all = corr_features_cat + corr_features_num
    df = df.drop(corr_features_all ,axis=1)

    outlier_remover = OutlierRemover()

    numeric_features = df_num.columns.tolist()
    numeric_transformer = Pipeline(
        steps=[('outlier_remover', OutlierRemover()),
               ("imputer", SimpleImputer(strategy="median")), # put back in because of feature selection even though RF can deal with NaNs
               ("scaler", StandardScaler(with_mean=True, with_std=True)),
               ('threshold', loc_filter),
               ('feature_selection', SelectFromModel(RandomForestClassifier(random_state=0), max_features=15)),
               #('feature_selection', SelectFromModel(LinearSVC(penalty="l1", dual = False), max_features=10)),
               #('SelectPercentile', SelectPercentile(mutual_info_classif, percentile=30)),
              ])

    categorical_features = df_cat.columns.tolist()
    categorical_transformer = Pipeline(
            steps=[ 
                    ("onehot", OneHotEncoder(drop='first')),
                    ('threshold', loc_filter),
                ]
            )

    preprocess = ColumnTransformer(
    transformers=[
                ("num", numeric_transformer, numeric_features),
                ("cat", categorical_transformer, categorical_features),
            ]
        )

    outcome_var = 'is_DN'

    y = df_target[[outcome_var]]
    y = y.replace({'False':0, 'True':1})

    X_pre = preprocess.fit_transform(df, y)

    print('number of features after fit_transform', X_pre.shape)

    # some ugly code which extracts the column names after filtering - how is there no off the shelf solution?!?
    all_cat_feat = preprocess.transformers_[1][1]["onehot"].get_feature_names(categorical_features).tolist()
    mask_cat = preprocess.transformers_[1][1]['threshold'].get_support().tolist()
    filt_cat_feat = list(compress(all_cat_feat, mask_cat))

    all_num_feat = preprocess.transformers_[0][2]
    mask_num = preprocess.transformers_[0][1]['feature_selection'].get_support().tolist()
    filt_num_feat = list(compress(all_num_feat, mask_num))

    # this is to get the scaling factors
    # add for if median is imputed
    imp_median = SimpleImputer(strategy="median")
    imp = imp_median.fit_transform(df[filt_num_feat])
    df_imp = pd.DataFrame(imp, columns = filt_num_feat)

    # crop the outliers
    outl = outlier_remover.fit_transform(df_imp[filt_num_feat])
    df_outl = pd.DataFrame(outl, columns = filt_num_feat)

    # get scaling factors
    stand_df = pd.DataFrame(df_outl.mean(), columns = ['mean']).T.append(pd.DataFrame(df_outl.std(), columns = ['std']).T)

    column_header = [outcome_var] + filt_num_feat + filt_cat_feat

    y_pre = y.to_numpy().reshape(len(y), 1)
    X = np.concatenate((y_pre, X_pre), axis=1)

    np.random.shuffle(X)

    train, validation = np.split(X, [int(0.8 * len(X))])
    
    local_directory = f'{base_dir}{suffix}{file_name_base}/'
    
    path = pathlib.Path(local_directory)
    path.mkdir(parents=True, exist_ok=True)
    
    pd.DataFrame(train).to_csv(f"{base_dir}{suffix}{file_name_base}/train.csv", header=False, index=False)
    pd.DataFrame(validation).to_csv(f"{base_dir}{suffix}{file_name_base}/validation.csv", header=False, index=False)
    pd.DataFrame(column_header).to_csv(f"{base_dir}{suffix}{file_name_base}/column_header.csv", header=False, index=False) 
    pd.DataFrame(stand_df).to_csv(f"{base_dir}{suffix}{file_name_base}/standardization.csv", header=True, index=True)
    
    print(file_name_base, 'saved')    

In [3]:
# define paths
base_dir = '/home/jovyan/work/Goldfinch/ML_paper/'
output_suffix = 'output_files/training_output/'
suffix = 'raw_data/intermediate_files/'

In [4]:
# one round
executionid = 1

preprocess_dataset(base_dir, executionid, suffix)

original (445, 8313)
after dropna (445, 350)
number of numeric features (445, 155)
number of categorical features (445, 195)
number of numeric features after correlation drop (445, 101)
number of categorical features after correlation drop (445, 30)
number of features after fit_transform (445, 37)
0_years_0 saved


In [5]:
! pip list

Package                           Version
--------------------------------- -----------
alembic                           1.8.1
altair                            4.2.0
anyio                             3.6.1
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
asttokens                         2.0.8
async-generator                   1.10
attrs                             22.1.0
Babel                             2.10.3
backcall                          0.2.0
backports.functools-lru-cache     1.6.4
beautifulsoup4                    4.11.1
bleach                            5.0.1
blinker                           1.4
bokeh                             2.4.3
Bottleneck                        1.3.5
brotlipy                          0.7.0
cached-property                   1.5.2
catboost                          1.1.1
certifi                           2022.9.24
certipy                           0.1.3
cffi                              1.15.1
charset-normalizer       